In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Parameter
import torch.nn.functional as F
import math

import TopicVAE

from sklearn.datasets import fetch_20newsgroups
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import argparse
from types import SimpleNamespace

import gensim.downloader as api
from gensim.models import KeyedVectors

In [6]:
newsgroups_train = fetch_20newsgroups(subset='train')

vectorizer = CountVectorizer(stop_words = 'english', min_df=.01, max_df=0.9, 
                             token_pattern = u'(?ui)\\b[a-z]{3,}\\b')
count_vecs = vectorizer.fit_transform(newsgroups_train.data)
doc_term_matrix = count_vecs.toarray()
doc_term_matrix.shape # number of documents, number of words (in vocab)
tokenizer = vectorizer.build_tokenizer()

# note: vectorizer.get_feature_names() != vectorizer.vocabulary_

doc_term_matrix_tensor = torch.from_numpy(doc_term_matrix).float()

args_dict = {"en1_units" : 100, "en2_units" : 100, "num_topic" : 50, 
             "batch_size" : 200, "optimizer" : 80, "learning_rate" : 0.002, 
             "momentum" : 0.99, "num_epoch" : 80, "init_mult" : 1, 
             "variance" : 0.995, "start" : True, "nogpu" : True, "embedding_dim" : 300}
args = SimpleNamespace(**args_dict)
args.num_input = doc_term_matrix_tensor.shape[1]

In [ ]:
# pretrained_language_model = api.load("fasttext-wiki-news-subwords-300")
# pretrained_language_model.save("fasttext-wiki-news-subwords-300")

In [3]:
EMBEDDING_FILE = "fasttext-wiki-news-subwords-300"
pretrained_language_model = KeyedVectors.load(EMBEDDING_FILE)

In [ ]:
pretrained_language_model.most_similar("cat")[0]

In [5]:
embedding_matrix = np.random.randn(doc_term_matrix.shape[1], 300)

iterator = 0
for word in vectorizer.get_feature_names():
    if word in pretrained_language_model.vocab:
        embedding_matrix[iterator] = pretrained_language_model.wv.word_vec(word)
    else:
        continue
        # embedding_matrix[iterator] = pretrained_language_model.wv.most_similar(word)
        # or something like that
    iterator += 1

/anaconda3/envs/math689env/lib/python3.7/site-packages/ipykernel/__main__.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [ ]:
sum(embedding_matrix.sum(1) == 0) # when it was np.zeros

In [ ]:
type(pretrained_language_model.wv.word_vec("cats"))

In [8]:
model_GSMLDA = TopicVAE.GSMLDA(args, embedding_matrix)
optimizer_GSMLDA = torch.optim.Adam(model_GSMLDA.parameters(), args.learning_rate, betas=(args.momentum, 0.999))
model_GSMLDA = train(model_GSMLDA, args, optimizer_GSMLDA, doc_term_matrix_tensor)




ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()